In [1]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
haskey(ENV, "LD_LIBRARY_PATH") && @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames",
    "FASTX",
    "XAM",
    "uCSV",
    "CodecZlib",
    "ProgressMeter",
    "StatsBase",
    "Statistics",
    "CSV",
    "Random",
    "Distributions",
    "Plots"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

  Activating new project at `/tmp/jl_sZYaJ9`
   Resolving package versions...
    Updating `/tmp/jl_sZYaJ9/Project.toml`
  [295af30f] + Revise v3.5.14
    Updating `/tmp/jl_sZYaJ9/Manifest.toml`
  [da1fd8a2] + CodeTracking v1.3.5
  [aa1ae85d] + JuliaInterpreter v0.9.32
  [6f1432cf] + LoweredCodeUtils v2.4.6
  [bac558e1] + OrderedCollections v1.6.3
  [ae029012] + Requires v1.3.0
  [295af30f] + Revise v3.5.14
  [0dad84c5] + ArgTools v1.1.1
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [ade2ca70] + Dates
  [8ba89e20] + Distributed
  [f43a241f] + Downloads v1.6.0
  [7b1f6079] + FileWatching
  [b77e0a4c] + InteractiveUtils
  [b27032c2] + LibCURL v0.6.4
  [76f85450] + LibGit2
  [8f399da3] + Libdl
  [56ddb016] + Logging
  [d6f4376e] + Markdown
  [ca575930] + NetworkOptions v1.2.0
  [44cfe95a] + Pkg v1.10.0
  [de0858da] + Printf
  [3fa0cd96] + REPL
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [6462fe0b] + Sockets
  [fa267f1f] + TOML v1.0.3
  [a4e569a6] + Ta

In [2]:
function zipf_distribution(n)
  return 1 ./ collect(1:n)
end

function dirichlet_distribution(alpha)
  return rand(Distributions.Dirichlet(alpha))
end

dirichlet_distribution (generic function with 1 method)

In [3]:
basedir = dirname(pwd())
data_dir = joinpath(basedir, "data")

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/metagenome/data"

In [4]:
@time ncbi_species_table = Mycelia.list_species();

 10.974639 seconds (27.40 M allocations: 1.486 GiB, 14.26% gc time, 5.59% compilation time: 6% of which was recompilation)


In [5]:
@time refseq_metadata_table = Mycelia.load_refseq_metadata();

 24.445105 seconds (55.64 M allocations: 3.614 GiB, 12.89% gc time, 42.91% compilation time: 1% of which was recompilation)


In [6]:
# function subsample(t, n)
#     return t[StatsBase.sample(1:DataFrames.nrow(t), N, ordered=true, replace=false), :]
# end

function sample_evenly_by_group(df, column, N)
    gdf = collect(DataFrames.groupby(df, column))
    return_df = DataFrames.DataFrame()
    while DataFrames.nrow(return_df) < N
        this_gdf = rand(gdf)
        row = rand(1:DataFrames.nrow(this_gdf))
        sampled_row = this_gdf[row, :]
        push!(return_df, sampled_row)
    end
    return return_df
end

N = 1
Random.seed!(N)

Random.TaskLocalRNG()

In [7]:
# e coli
# ec_taxa_ids = Set(Mycelia.list_subtaxa(562))
# ec_metadata_table = refseq_metadata_table[map(x -> x in ec_taxa_ids, refseq_metadata_table[!, "species_taxid"]), :]
# 37891 distinct accessions
ec_metadata_table = refseq_metadata_table[refseq_metadata_table[!, "species_taxid"] .== 562, :]
ec_metadata_table = sample_evenly_by_group(ec_metadata_table, "taxid", N)
# 2271 distinct taxids
# sort(collect(StatsBase.countmap(ec_metadata_table[!, "taxid"])), by=x->x[2], rev=true)
# ec_metadata_table = subsample(ec_metadata_table, N)

Row,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,Date,String,String,String,String,String,String,String,String,String,String,Int64,Int64,Float64,Int64,Int64,Int64,String,String,Date,Int64,Int64,Int64,String
1,GCF_003627195.1,PRJNA224116,SAMN10139565,na,na,511145,562,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,na,latest,Complete Genome,Major,Full,2018-10-08,ASM362719v1,None,GCA_003627195.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/003/627/195/GCF_003627195.1_ASM362719v1,na,na,na,haploid,bacteria,4639694,4639694,51.0,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2024-03-06,4532,4219,123,na


In [8]:
# staph
# sa_taxa_ids = Set(Mycelia.list_subtaxa(1280))
# 16411 distinct accessions
sa_metadata_table = refseq_metadata_table[refseq_metadata_table[!, "species_taxid"] .== 1280, :]
# 4109 distinct taxids
# sort(collect(StatsBase.countmap(sa_metadata_table[!, "taxid"])), by=x->x[2], rev=true)
sa_metadata_table = sample_evenly_by_group(sa_metadata_table, "taxid", N)

Row,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,Date,String,String,String,String,String,String,String,String,String,String,Int64,Int64,Float64,Int64,Int64,Int64,String,String,Date,Int64,Int64,Int64,String
1,GCF_000561005.1,PRJNA224116,SAMN02383712,JDKJ00000000.1,na,1410792,1280,Staphylococcus aureus S72396,strain=S72396,na,latest,Scaffold,Major,Full,2014-02-06,Stap_aure_S72396_V1,Broad Institute,GCA_000561005.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/561/005/GCF_000561005.1_Stap_aure_S72396_V1,na,na,na,haploid,bacteria,2865640,2864782,33.0,0,63,67,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-10-30,2919,2745,74,na


In [9]:
# pseudomonas
# pa_taxa_ids = Set(Mycelia.list_subtaxa(287))
# 9555 distinct accessions
pa_metadata_table = refseq_metadata_table[refseq_metadata_table[!, "species_taxid"] .== 287, :]
# 248 distinct taxids
# sort(collect(StatsBase.countmap(pa_metadata_table[!, "taxid"])), by=x->x[2], rev=true)
pa_metadata_table = sample_evenly_by_group(pa_metadata_table, "taxid", N)

Row,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,Date,String,String,String,String,String,String,String,String,String,String,Int64,Int64,Float64,Int64,Int64,Int64,String,String,Date,Int64,Int64,Int64,String
1,GCF_000496605.2,PRJNA224116,SAMN02603191,na,na,1279007,287,Pseudomonas aeruginosa PA1,strain=PA1,na,latest,Complete Genome,Major,Full,2015-09-11,ASM49660v2,"Department of Microbiology, The Third Military Medical University",GCA_000496605.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/496/605/GCF_000496605.2_ASM49660v2,na,na,na,haploid,bacteria,6498072,6498072,66.5,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-12-08,6067,5937,81,26659688


In [10]:
# taxa_ids_of_interest = Set(union(ec_taxa_ids, sa_taxa_ids, pa_taxa_ids))
# accessions_of_interest_metadata_table = refseq_metadata_table[map(x -> x in taxa_ids_of_interest, refseq_metadata_table[!, "taxid"]) .| map(x -> x in taxa_ids_of_interest, refseq_metadata_table[!, "species_taxid"]), :]

In [11]:
accessions_of_interest_metadata_table = vcat(ec_metadata_table, sa_metadata_table, pa_metadata_table)

Row,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,Date,String,String,String,String,String,String,String,String,String,String,Int64,Int64,Float64,Int64,Int64,Int64,String,String,Date,Int64,Int64,Int64,String
1,GCF_003627195.1,PRJNA224116,SAMN10139565,na,na,511145,562,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,na,latest,Complete Genome,Major,Full,2018-10-08,ASM362719v1,None,GCA_003627195.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/003/627/195/GCF_003627195.1_ASM362719v1,na,na,na,haploid,bacteria,4639694,4639694,51.0,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2024-03-06,4532,4219,123,na
2,GCF_000561005.1,PRJNA224116,SAMN02383712,JDKJ00000000.1,na,1410792,1280,Staphylococcus aureus S72396,strain=S72396,na,latest,Scaffold,Major,Full,2014-02-06,Stap_aure_S72396_V1,Broad Institute,GCA_000561005.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/561/005/GCF_000561005.1_Stap_aure_S72396_V1,na,na,na,haploid,bacteria,2865640,2864782,33.0,0,63,67,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-10-30,2919,2745,74,na
3,GCF_000496605.2,PRJNA224116,SAMN02603191,na,na,1279007,287,Pseudomonas aeruginosa PA1,strain=PA1,na,latest,Complete Genome,Major,Full,2015-09-11,ASM49660v2,"Department of Microbiology, The Third Military Medical University",GCA_000496605.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/496/605/GCF_000496605.2_ASM49660v2,na,na,na,haploid,bacteria,6498072,6498072,66.5,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-12-08,6067,5937,81,26659688


In [12]:
accessions_of_interest_metadata_table[!, "out_directory"] .= ""
ProgressMeter.@showprogress for (i, accession) in enumerate(accessions_of_interest_metadata_table[!, "#assembly_accession"])
    # @show accession
    # Mycelia.ncbi_genome_download_accession
    accessions_of_interest_metadata_table[i, "out_directory"] = Mycelia.ncbi_genome_download_accession(
        accession=accession,
        outdir = joinpath(data_dir, "ncbi-genomes")
        # include_string="gbff"
    )
end
accessions_of_interest_metadata_table[!, "out_directory"]

New version of client (16.22.1) available at https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/datasets.
New version of client (16.22.1) available at https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/datasets.
Progress: 100%|█████████████████████████████████████████| Time: 0:00:12


3-element Vector{String}:
 "/global/cfs/cdirs/m4269/cjprybo" ⋯ 66 bytes ⋯ "bi_dataset/data/GCF_003627195.1"
 "/global/cfs/cdirs/m4269/cjprybo" ⋯ 66 bytes ⋯ "bi_dataset/data/GCF_000561005.1"
 "/global/cfs/cdirs/m4269/cjprybo" ⋯ 66 bytes ⋯ "bi_dataset/data/GCF_000496605.2"

In [13]:
# I can use the LOCUS line in the genbank files to specify genome topology, but don't bother with that right now

In [14]:
# read in each genome, and tag the relative abundance (and eventually, topology) in the joined fasta files

In [15]:
n_genomes = DataFrames.nrow(accessions_of_interest_metadata_table)

3

In [16]:
# dirichlet_distribution(zipf_distribution(10^3))
raw_abundances = zipf_distribution(n_genomes)
normalized_abundances = raw_abundances ./ sum(raw_abundances)

3-element Vector{Float64}:
 0.5454545454545455
 0.27272727272727276
 0.18181818181818182

In [17]:
accessions_of_interest_metadata_table

Row,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id,out_directory
,String,String,String,String,String,Int64,Int64,String,String,String,String,String,String,String,Date,String,String,String,String,String,String,String,String,String,String,Int64,Int64,Float64,Int64,Int64,Int64,String,String,Date,Int64,Int64,Int64,String,String
1,GCF_003627195.1,PRJNA224116,SAMN10139565,na,na,511145,562,Escherichia coli str. K-12 substr. MG1655,strain=K-12 substr. MG1655,na,latest,Complete Genome,Major,Full,2018-10-08,ASM362719v1,None,GCA_003627195.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/003/627/195/GCF_003627195.1_ASM362719v1,na,na,na,haploid,bacteria,4639694,4639694,51.0,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2024-03-06,4532,4219,123,na,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/metagenome/data/ncbi-genomes/GCF_003627195.1/ncbi_dataset/data/GCF_003627195.1
2,GCF_000561005.1,PRJNA224116,SAMN02383712,JDKJ00000000.1,na,1410792,1280,Staphylococcus aureus S72396,strain=S72396,na,latest,Scaffold,Major,Full,2014-02-06,Stap_aure_S72396_V1,Broad Institute,GCA_000561005.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/561/005/GCF_000561005.1_Stap_aure_S72396_V1,na,na,na,haploid,bacteria,2865640,2864782,33.0,0,63,67,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-10-30,2919,2745,74,na,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/metagenome/data/ncbi-genomes/GCF_000561005.1/ncbi_dataset/data/GCF_000561005.1
3,GCF_000496605.2,PRJNA224116,SAMN02603191,na,na,1279007,287,Pseudomonas aeruginosa PA1,strain=PA1,na,latest,Complete Genome,Major,Full,2015-09-11,ASM49660v2,"Department of Microbiology, The Third Military Medical University",GCA_000496605.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/496/605/GCF_000496605.2_ASM49660v2,na,na,na,haploid,bacteria,6498072,6498072,66.5,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023-12-08,6067,5937,81,26659688,/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/metagenome/data/ncbi-genomes/GCF_000496605.2/ncbi_dataset/data/GCF_000496605.2


In [18]:
accessions_of_interest_metadata_table[!, "relative_abundances"] = normalized_abundances

3-element Vector{Float64}:
 0.5454545454545455
 0.27272727272727276
 0.18181818181818182

In [19]:
accession_dir = first(accessions_of_interest_metadata_table[!, "out_directory"])

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/metagenome/data/ncbi-genomes/GCF_003627195.1/ncbi_dataset/data/GCF_003627195.1"

In [20]:
accession_fasta = first(filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(accession_dir, join=true)))

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/metagenome/data/ncbi-genomes/GCF_003627195.1/ncbi_dataset/data/GCF_003627195.1/GCF_003627195.1_ASM362719v1_genomic.fna"

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

...

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found